## **1. Import necessary libraries**

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.vhtQ46Rj2w/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.rEb6XjbBGw/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.cLfdl2mI4J/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [2]:
import pandas as pd
import os
import requests
import time
import random

from io import BytesIO
from PIL import Image
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## **2. Crawl**

In [3]:
# Initialize google chrome browser
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless=new')
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome(
    'chromedriver',
    options=chrome_options
)

# Create a folder for storing articles
root_dir = './vn_news_corpus'
os.makedirs(root_dir, exist_ok=True)
n_pages = 10 # Change if you want more articles
article_id = 0

for page_idx in tqdm(range(n_pages)):
    # Access to table page
    main_url = f'https://vietnamnet.vn/thoi-su-page{page_idx}' 
    driver.get(main_url)

    # Get list of articles (list of URLs)
    news_lst_xpath = '//div[@class="topStory-15nd"]/div/div[1]/a'
    news_tags = driver.find_elements(
        By.XPATH, 
        news_lst_xpath
    )
    news_page_urls = [
        news_tag.get_attribute('href') \
            for news_tag in news_tags
    ]

    for news_page_url in news_page_urls:
        # Access to article page
        driver.get(news_page_url)
        time.sleep(1)
    
        # Try to get main content tag
        main_content_xpath = '//div[@class="content-detail"]'
        try:
            main_content_tag = driver.find_element(
                By.XPATH, 
                main_content_xpath
            )
        except:
            continue
        # Ignore video article 
        video_content_xpath = '//div[@class="video-detail"]'
        try:
            video_content_tag = main_content_tag.find_element(
                By.XPATH,
                video_content_xpath
            )
            continue
        except:
            pass
        # Get title (h1 tag)
        title = main_content_tag.find_element(
            By.TAG_NAME,
            'h1'
        ).text.strip()
        # Get abstract (h2 tag)
        abstract = main_content_tag.find_element(
            By.TAG_NAME,
            'h2'
        ).text.strip()
        # Get author name (span tag)
        try:
            author_xpath = '//span[@class="name"]'
            author = main_content_tag.find_element(
                By.XPATH,
                author_xpath
            ).text.strip()
        except:
            author = ''
        # Get paragraphs (all p tags in div "maincontent main-content")
        paragraphs_xpath = '//div[@class="maincontent main-content"]/p'
        paragraphs_tags = main_content_tag.find_elements(
            By.XPATH,
            paragraphs_xpath
        )
        paragraphs_lst = [
            paragraphs_tag.text.strip() \
                for paragraphs_tag in paragraphs_tags
        ]
        paragraphs = ' '.join(paragraphs_lst)
        # Combine title, abstract, authoor and paragraphs 
        final_content_lst = [title, abstract, paragraphs, author]
        final_content = '\n\n'.join(final_content_lst)

        # Save artile to .txt file
        article_filename = f'article_{article_id:05d}.txt'
        article_savepath = os.path.join(
            root_dir,
            article_filename
        )
        article_id += 1
        with open(article_savepath, 'w') as f:
            f.write(final_content)
        
        # Move back to previous page
        driver.back()

100%|██████████| 10/10 [21:20<00:00, 128.00s/it]


In [4]:
!zip -r vn_news_corpus.zip vn_news_corpus

  adding: vn_news_corpus/ (stored 0%)
  adding: vn_news_corpus/article_00086.txt (deflated 59%)
  adding: vn_news_corpus/article_00097.txt (deflated 63%)
  adding: vn_news_corpus/article_00065.txt (deflated 65%)
  adding: vn_news_corpus/article_00028.txt (deflated 52%)
  adding: vn_news_corpus/article_00030.txt (deflated 66%)
  adding: vn_news_corpus/article_00026.txt (deflated 53%)
  adding: vn_news_corpus/article_00006.txt (deflated 62%)
  adding: vn_news_corpus/article_00123.txt (deflated 63%)
  adding: vn_news_corpus/article_00008.txt (deflated 56%)
  adding: vn_news_corpus/article_00069.txt (deflated 34%)
  adding: vn_news_corpus/article_00009.txt (deflated 57%)
  adding: vn_news_corpus/article_00047.txt (deflated 63%)
  adding: vn_news_corpus/article_00031.txt (deflated 48%)
  adding: vn_news_corpus/article_00023.txt (deflated 55%)
  adding: vn_news_corpus/article_00110.txt (deflated 61%)
  adding: vn_news_corpus/article_00072.txt (deflated 68%)
  adding: vn_news_corpus/article_0

In [5]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
!cp '/content/vn_news_corpus.zip' '/content/gdrive/MyDrive/TRUONG_UIT/AI VIETNAM/Crawling_Data_Project'